In [ ]:
%%capture
pip install pycaret

In [ ]:
%%capture
pip install --upgrade scipy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pycaret.classification import *

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
train_df = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_df = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df.info()

# Numeric Columns Visualizations

In [ ]:
numerical_cols = train_df.select_dtypes(include=['float64']).columns
# ENG = Log transformation application function / TR= Log dönüşümü uygulama fonksiyonu
def plot_log_transformed_histogram(df, column, ax):
    # ENG = Since zero values will cause problems for log transformation, we add 1 / TR= Sıfır değerler log dönüşümü için problem oluşturacağından, 1 ekliyoruz.
    data_to_plot = df[column] + 1
    sns.histplot(np.log(data_to_plot), kde=True, ax=ax)
    ax.set_title(f'Log Transformed {column} Distribution')
    ax.set_xlabel(f'Log of {column}')
    ax.set_ylabel('Frequency')

# ENG= We will use subplots to plot the histograms for both columns side by side / TR= Her iki sütun için histogramları yan yana çizmek için subplots kullanacağız
fig, axes = plt.subplots(nrows=int(len(numerical_cols)/2), ncols=2, figsize=(14, 4 * int(len(numerical_cols)/2)))
axes = axes.flatten()  # Straightening axes / Eksenleri düzleştirme

# We draw a normal histogram for the 'Age' column / 'Age' sütunu için normal histogram çizdiriyoruz
sns.histplot(train_df['Age'], kde=True, ax=axes[0])
axes[0].set_title('Age Distribution')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')

# For other numerical columns, we plot a log-transformed histogram. / Diğer sayısal sütunlar için log dönüşümü uygulanmış histogram çizdiriyoruz
for i, col in enumerate(numerical_cols[1:], start=1):
    plot_log_transformed_histogram(train_df, col, axes[i])

plt.tight_layout()
plt.show()

#### ENG

* Based on the histogram, I have decided to fill the missing values with median values. Additionally, it can be observed that there is a very high frequency of zero expenditures in the data. This indicates that people generally do not spend a lot

* Therefore, filling these cells with median (0) will help maintain the accuracy of the classification model.

#### TR

* Histograma bakarak boş verileri median değerler ile doldurma kararı aldım. Ayrıca verilerde çok yüksek oranda 0 harcama olduğu görülmektedir. Bu durum, insanların genellikle çok fazla harcama yapmadığını göstermektedir.

* Bu nedenle, bu hücrelerin boş hücrelerini medyan (0) ile doldurmak, sınıflandırma modelinin doğruluğunu korumaya yardımcı olacaktır.

In [ ]:
train_df.info()

# Dealing with empty data

In [ ]:
def fill_missing_values(df):
    """
    -ENG=The provided DataFrame fills in empty data. It uses the median for numerical columns and
    the mode (the most frequently encountered value) for categorical columns.
   
    -TR=Fonksiyona sağlanan DataFrame'in boş verileri doldurur. Sayısal sütunlar için medyanı,
    kategorik sütunlar için mod kullanır (en sık karşılaşılan değerler).
    """

    # Detecting numerical and categorical columns / Sayısal ve kategorik sütunları bulma
    numerical_cols = df.select_dtypes(include=['float64']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns

    # Filling missing values in numerical columns with the median / Sayısal sütunlardaki eksik değerleri medyanla doldurma
    df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

    # Filling missing values in categorical columns with the mode / Kategorik sütunlardaki eksik değerleri modla doldurma
    df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

    return df.isna().sum()

In [ ]:
fill_missing_values(train_df)

In [ ]:
fill_missing_values(test_df)

# Feature Engineering

In [ ]:
def feature_engineering(df):
    #  *----PassengerId----*
    """
     -ENG=I will extract the first part of this column (group identifier) to calculate family size or group
    size using the 'PassengerId' column.
    
     -TR='PassengerId' sütununu kullanarak aile boyutunu veya grup büyüklüğünü hesaplamak için bu 
    sütunun ilk kısmını (grup kimliği) ayıracağım.
    """
    #We split the values in the 'PassengerId' column by the '_' character and take the first part.
    df['GroupID'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
    df['GroupID'] = df['GroupID'].astype(int)

    #We are calculating the group size based on Group ID.
    group_size = df.groupby('GroupID').size()
    
    # We are adding the calculated group size to the original dataset.
    df['GroupSize'] = df['GroupID'].apply(lambda x: group_size[x])
    
    df = df.drop('PassengerId',axis=1)
    
    #*----Cabin----*
    """
    Cabin = deck/num/side
    
     -ENG=I will split the 'Cabin' column by the '/' character and place it in separate columns I will
     also determine how many people are on deck
     -TR=Cabin sütununu '/' karakterine göre ayırıp ayrı ayrı sütunlara yerleştireceğim ayrıca güvertede kaç
     kişi olduğunu belirleyeceğim
    """
    df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
    df['Num'] = df['Num'].astype(int)
    deck_size = df.groupby('Deck').size()
    df['DeckSize'] = df['Deck'].apply(lambda x: deck_size[x])

    df = df.drop('Cabin',axis=1)
    
    #*----RoomService, FoodCourt, ShoppingMall, Spa, VRDeck----*
    """
    -ENG=I will create a new column that represents the total spending of passengers by summing
    the 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', and 'VRDeck' columns.
    -Tr=Yolcuların toplam harcamasını bulabilmek için 'RoomService', 'FoodCourt', 'ShoppingMall',
    'Spa', ve 'VRDeck' sütunlarını toplayarak yeni bir stün oluşturacağım 
    """
    df['TotalSpending'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    
    #*----Age----*
    """
    Child (0-12 years), Youth (12-18 years), Adult (18-60 years) and Elderly (60-120 years).
    
    -ENG=I will define the age groups
    -TR=Yaş gruplarını tanımlayacağım
    """
    bins = [0, 12, 18, 60, 120]
    labels = ['Child', 'Youth', 'Adult', 'Elderly']
    
    df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
    
    #*----Eng=I will use the additional columns I created from the raw data set to create new columns----*
    #*-------Tr=Ham veri setinden oluşturduğum ek sütunları kullanarak yeni sütunlar oluşturacağım-------*
    
    #*----TotalSpending----*
    """
    -Eng=I will categorize the amount spent (I made TotalSpending.describe to find out the intervals)
    -Tr=Harcanan Miktarın Kategorize Edeceğim (Aralıkları öğrenmek içibn TotalSpending.describe yaptım)
    """
    bins = [0, 716, 1441, 2803, df['TotalSpending'].max()]
    labels = ['Very Low', 'Low', 'Medium', 'High']
    
    df['SpendingCategory'] = pd.cut(df['TotalSpending'], bins=bins, labels=labels, include_lowest=True)
    
    #*----RoomService, FoodCourt, ShoppingMall, Spa, VRDeck----*
    """
    -Eng=I will find the category in which the person spends the most and how many categories they spend in.
    -Tr=Kişinin en çok harcama yaptığı kategoriyi ve kaç tane kategoride harcama yapmış bunları bulacağım
    """
    df['MostSpentOn'] = df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].idxmax(axis=1)
    df['MostSpentOn'] = df.apply(lambda row: row['MostSpentOn'] if row[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum() > 0 else 'None', axis=1)
    
    df['NumberOfSpendingCategories'] = (df[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] > 0).sum(axis=1)

    
    #gender-guesser denene bilir belki
    df=df.drop("Name",axis=1)
    
    return df

In [ ]:
train_df = feature_engineering(train_df)

In [ ]:
test_df = feature_engineering(test_df)

In [ ]:
train_df.sample(5)

In [ ]:
train_df.shape

# Converting Categorical and Bool Columns to Numeric Columns

In [ ]:
train_df.info()

In [ ]:
def encode_categorical_columns(df):
    """
    Encodes categorical columns in the given DataFrame using LabelEncoder. 
    This function identifies columns of 'object' and 'bool' types and converts them to numerical values.
    """
    label_encoder = LabelEncoder()
    categorical_cols = df.select_dtypes(include=['object', 'bool','category']).columns
    # Applying LabelEncoder to each categorical column
    df[categorical_cols] = df[categorical_cols].apply(lambda col: label_encoder.fit_transform(col))

    return categorical_cols, df

In [ ]:
encode_categorical_columns(train_df)

In [ ]:
encode_categorical_columns(test_df)

# Correlation Table

In [ ]:
plt.figure(figsize=(14,12))
sns.heatmap(train_df.corr(),annot=True, fmt='.2f', linewidths=.5)
plt.title("Correlation Table")
plt.show()

In [ ]:
train_df.corr()["Transported"].sort_values(ascending=False)

# PYCARET

In [ ]:
setup(train_df, target="Transported")

In [ ]:
best_model=compare_models()

# Submission

In [ ]:
preded_col = best_model.predict(test_df)
preded_col

In [ ]:
preded_col =preded_col.astype(bool)

In [ ]:
sample_submission["Transported"] = preded_col

In [ ]:
sample_submission.to_csv("submission.csv",index=False)